In [8]:
# ======================================= 
# ECC ElGamal Encryption for "PARIS" (Original + Offset, offset not printed in ciphertext)
# =======================================

p = 89
a = 22
b = 28

# =======================================
# Elliptic Curve Operations
# =======================================

def inv_mod(k, p):
    return pow(k, p-2, p)

def ec_add(P, Q):
    if P is None: return Q
    if Q is None: return P

    (x1, y1) = P
    (x2, y2) = Q

    if x1 == x2 and y1 == (-y2 % p):
        return None

    if P != Q:
        m = ((y2 - y1) * inv_mod(x2 - x1, p)) % p
    else:
        m = ((3*x1*x1 + a) * inv_mod(2*y1, p)) % p

    x3 = (m*m - x1 - x2) % p
    y3 = (m*(x1 - x3) - y1) % p
    return (x3, y3)

def ec_scalar_mul(k, P):
    R = None
    Q = P
    while k > 0:
        if k & 1:
            R = ec_add(R, Q)
        Q = ec_add(Q, Q)
        k >>= 1
    return R

# =======================================
# Square root modulo p (brute force)
# =======================================

def sqrt_mod(a, p):
    for x in range(p):
        if (x*x) % p == a:
            return x
    return None

# =======================================
# Encode letter with offset (A=0)
# =======================================

def encode_letter(ch):
    m = ord(ch.upper()) - 65   # A=0, B=1, ..., Z=25
    offset = 0
    while offset < 200:
        x = (m + offset) % p
        rhs = (x*x*x + a*x + b) % p
        y = sqrt_mod(rhs, p)
        if y is not None:
            if offset == 0:
                # Original number corresponds to a valid point
                print(f"The letter {ch} corresponds to the number {m}. "
                      f"Now, the point ({x}, {y}) belongs to E. Thus, {ch} ←→ (({x}, {y}), {offset}).")
            else:
                # Original number did NOT belong; needed offset
                skipped = ", ".join(str(m + i) for i in range(offset))
                print(f"The letter {ch} corresponds to the number {m}. "
                      f"There are no points in E with x = {skipped}. "
                      f"However, ({x}, {y}) ∈ E. Thus, {ch} ←→ (({x}, {y}), {offset}).")
            return (x, y), offset
        offset += 1
    raise ValueError(f"Encoding failed for letter {ch}")
    

# =======================================
# Parameters
# =======================================

R = (17, 8)     # Base point
eB = (3, 78)    # Bob's public key

# =======================================
# Recover Bob's private key
# =======================================

dB = None
for cand in range(1, p):
    if ec_scalar_mul(cand, R) == eB:
        dB = cand
        break

print("\nBob’s private key =", dB)

# =======================================
# Encrypt message
# =======================================

message = "PARIS"
encoded = [encode_letter(ch) for ch in message]

k = 23
r = ec_scalar_mul(k, R)
kQ = ec_scalar_mul(k, eB)

cipher = []
for (Pm, off) in encoded:
    C2 = ec_add(Pm, kQ)
    cipher.append(C2)  # Only store points, discard offset

# =======================================
# Print ciphertext without offset
# =======================================

print("\nThe encrypted message is\n")
print("y = ", end="")
print("; ".join([f"({C2[0]}, {C2[1]})" for C2 in cipher]))
print(f"r = ({r[0]}, {r[1]})")



Bob’s private key = 41
The letter P corresponds to the number 15. Now, the point (15, 23) belongs to E. Thus, P ←→ ((15, 23), 0).
The letter A corresponds to the number 0. There are no points in E with x = 0, 1. However, (2, 13) ∈ E. Thus, A ←→ ((2, 13), 2).
The letter R corresponds to the number 17. Now, the point (17, 8) belongs to E. Thus, R ←→ ((17, 8), 0).
The letter I corresponds to the number 8. Now, the point (8, 2) belongs to E. Thus, I ←→ ((8, 2), 0).
The letter S corresponds to the number 18. There are no points in E with x = 18, 19, 20. However, (21, 36) ∈ E. Thus, S ←→ ((21, 36), 3).

The encrypted message is

y = (75, 64); (70, 82); (15, 23); (65, 19); (70, 7)
r = (36, 22)


In [11]:
# =======================================
# ECC ElGamal Decryption with Offset
# =======================================

p = 89
a = 22
b = 28

# ECC operations
def inv_mod(k, p):
    return pow(k, p-2, p)

def ec_add(P, Q):
    if P is None: return Q
    if Q is None: return P

    (x1, y1) = P
    (x2, y2) = Q

    if x1 == x2 and y1 == (-y2 % p):
        return None

    if P != Q:
        m = ((y2 - y1) * inv_mod(x2 - x1, p)) % p
    else:
        m = ((3*x1*x1 + a) * inv_mod(2*y1, p)) % p

    x3 = (m*m - x1 - x2) % p
    y3 = (m*(x1 - x3) - y1) % p
    return (x3, y3)

def ec_scalar_mul(k, P):
    R = None
    Q = P
    while k > 0:
        if k & 1:
            R = ec_add(R, Q)
        Q = ec_add(Q, Q)
        k >>= 1
    return R

def ec_neg(P):
    x, y = P
    return (x, (-y) % p)

# Decode point to letter using x and offset
def decode_point_with_offset(x, offset):
    # Subtract offset and map to A=0, B=1, ...
    num = (x - offset) % p
    letter = chr((num % 26) + 65)
    return letter

# =======================================
# Received data
# =======================================

cipher_with_offsets = [
    ((75, 64), 0),
    ((70, 82), 2),
    ((15, 23), 0),
    ((65, 19), 0),
    ((70, 7), 3)
]
r = (36, 22)
eB = (3, 78)

# Bob's private key
dB = 41

# Compute shared mask: S = dB * r
S = ec_scalar_mul(dB, r)

# Decrypt each block
decrypted_points = []
decrypted_message = ""
for (C2, offset) in cipher_with_offsets:
    Pm = ec_add(C2, ec_neg(S))  # Decrypt point
    decrypted_points.append(Pm)
    letter = decode_point_with_offset(Pm[0], offset)  # Apply offset to recover original letter
    decrypted_message += letter

# Print results
print("Decrypted points:", decrypted_points)
print("Decrypted message:", decrypted_message)


Decrypted points: [(15, 23), (2, 13), (17, 8), (8, 2), (21, 36)]
Decrypted message: PARIS
